# Import Required Libraries
import numpy as np

In [1]:
# Import Required Libraries
import numpy as np

# Question 1: Optimized Edit Distance with Gap Penalty

In [2]:
def edit_distance_gap(seq1, seq2, gap_penalty_consecutive, gap_penalty_isolated):
    n = len(seq1)
    m = len(seq2)

    dp = [[[float('inf'), False] for _ in range(m + 1)] for _ in range(n + 1)]

    dp[0][0] = (0, False)

    for j in range(1, m + 1):
        if j == 1:
            dp[0][j] = (gap_penalty_isolated, True)
        else:
            dp[0][j] = (dp[0][j - 1][0] + gap_penalty_consecutive, True)

    for i in range(1, n + 1):
        if i == 1:
            dp[i][0] = (gap_penalty_isolated, True)
        else:
            dp[i][0] = (dp[i - 1][0][0] + gap_penalty_consecutive, True)

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i - 1] == seq2[j - 1]:
                cost_match = dp[i - 1][j - 1][0]
            else:
                cost_match = dp[i - 1][j - 1][0] + 1

            if dp[i][j - 1][1]:
                cost_insert = dp[i][j - 1][0] + gap_penalty_consecutive
            else:
                cost_insert = dp[i][j - 1][0] + gap_penalty_isolated

            if dp[i - 1][j][1]:
                cost_delete = dp[i - 1][j][0] + gap_penalty_consecutive
            else:
                cost_delete = dp[i - 1][j][0] + gap_penalty_isolated

            min_cost = min(cost_match, cost_insert, cost_delete)
            if min_cost == cost_match:
                dp[i][j] = (cost_match, False)
            elif min_cost == cost_insert:
                dp[i][j] = (cost_insert, True)
            else:
                dp[i][j] = (cost_delete, True)

    return dp[n][m][0]

# Question 2: Hybrid Alignment using Needleman-Wunsch and Smith-Waterman

In [3]:
def hybrid_alignment(seq1, seq2, gap_penalty_consecutive, gap_penalty_isolated, match_gain, mismatch_loss):
    matrix = [[[0, False] for _ in range(len(seq2) + 1)] for _ in range(len(seq1) + 1)]

    traceback = [[None for _ in range(len(seq2) + 1)] for _ in range(len(seq1) + 1)]

    matrix[0][0] = [0, False]
    for i in range(1, len(seq1) + 1):
        matrix[i][0] = [gap_penalty_consecutive + matrix[i - 1][0][0], True]
        traceback[i][0] = (i - 1, 0)
    for j in range(1, len(seq2) + 1):
        matrix[0][j] = [gap_penalty_consecutive + matrix[0][j - 1][0], True]
        traceback[0][j] = (0, j - 1)

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if seq1[i - 1] == seq2[j - 1]:
                match = match_gain + matrix[i - 1][j - 1][0]
                gap1 = gap_penalty_consecutive + matrix[i][j - 1][0] if matrix[i][j - 1][1] else gap_penalty_isolated + matrix[i][j - 1][0]
                gap2 = gap_penalty_consecutive + matrix[i - 1][j][0] if matrix[i - 1][j][1] else gap_penalty_isolated + matrix[i - 1][j][0]

                max_here = max(match, gap1, gap2)
                if max_here == match:
                    matrix[i][j] = [match, False]
                    traceback[i][j] = (i - 1, j - 1)
                elif max_here == gap1:
                    matrix[i][j] = [gap1, True]
                    traceback[i][j] = (i, j - 1)
                else:
                    matrix[i][j] = [gap2, True]
                    traceback[i][j] = (i - 1, j)
            else:
                mismatch = mismatch_loss + matrix[i - 1][j - 1][0]
                gap1 = gap_penalty_consecutive + matrix[i][j - 1][0] if matrix[i][j - 1][1] else gap_penalty_isolated + matrix[i][j - 1][0]
                gap2 = gap_penalty_consecutive + matrix[i - 1][j][0] if matrix[i - 1][j][1] else gap_penalty_isolated + matrix[i - 1][j][0]

                max_here = max(mismatch, gap1, gap2)
                if max_here == mismatch:
                    matrix[i][j] = [mismatch, False]
                    traceback[i][j] = (i - 1, j - 1)
                elif max_here == gap1:
                    matrix[i][j] = [gap1, True]
                    traceback[i][j] = (i, j - 1)
                else:
                    matrix[i][j] = [gap2, True]
                    traceback[i][j] = (i - 1, j)

            if matrix[i][j][0] < 0:
                matrix[i][j] = [0, False]
                traceback[i][j] = None

    aligned_seq1 = []
    aligned_seq2 = []
    i, j = len(seq1), len(seq2)
    while i > 0 or j > 0:
        if traceback[i][j] is None:
            break
        prev_i, prev_j = traceback[i][j]
        if prev_i == i - 1 and prev_j == j - 1:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
        elif prev_i == i - 1:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
        i, j = prev_i, prev_j

    aligned_seq1.reverse()
    aligned_seq2.reverse()

    optimal_score = matrix[len(seq1)][len(seq2)][0]

    return optimal_score, ''.join(aligned_seq1), ''.join(aligned_seq2)

# Question 3: Burrows-Wheeler Transformation and Alignment Preprocessing

In [4]:
def bwt_transform(seq):
    rotations = [seq[i:] + seq[:i] for i in range(len(seq))]
    sorted_rotations = sorted(rotations)
    bwt = ''.join([rotation[-1] for rotation in sorted_rotations])
    suffix_array = [rotations.index(rotation) for rotation in sorted_rotations]
    return bwt, suffix_array, sorted_rotations

def last_to_first_mapping(bwt, suffix_array):
    sorted_bwt = sorted(bwt)
    first_column = sorted_bwt
    last_to_first = []
    for i, char in enumerate(bwt):
        row_in_first_column = first_column.index(char)
        first_column[row_in_first_column] = None  # Mark it as used
        last_to_first.append(suffix_array[row_in_first_column])
    return last_to_first

def find_exact_matches(bwt, suffix_array, last_to_first, patterns, original_sequence):
    results = {}
    for pattern in patterns:
        L = [len(bwt)] * len(pattern)
        for i in range(len(pattern) - 1, -1, -1):
            c = pattern[i]
            L = [index for index in last_to_first if bwt[index] == c]
        results[pattern] = [suffix_array[i] for i in L]
    return results

# Question 4: Integrating Alignment with Compression

In [5]:
def smith_waterman_with_compression(seq1, seq2):
    compressed_seq1, suffix_array, rotations = bwt_transform(seq1)

    optimal_score, aligned_seq1, aligned_seq2 = hybrid_alignment(
        compressed_seq1, seq2,
        gap_penalty_consecutive=-2, gap_penalty_isolated=-3, match_gain=3, mismatch_loss=-1
    )

    decompressed_seq1 = ''.join([rotations[suffix_array.index(i)] for i in range(len(seq1)) if compressed_seq1[i] != '$'])

    return optimal_score, decompressed_seq1, aligned_seq2

# Code Execution

In [6]:


print("Executing optimized edit distance with gap penalties...")
seq1 = input("Enter the first sequence: ")
seq2 = input("Enter the second sequence: ")
gap_penalty_consecutive = int(input("Enter the gap penalty for consecutive gaps: "))
gap_penalty_isolated = int(input("Enter the gap penalty for isolated gaps: "))
result = edit_distance_gap(seq1, seq2, gap_penalty_consecutive, gap_penalty_isolated)
print("The minimum edit distance with gap penalties is:", result)

print("------------------------------------------------------------")

print("Executing hybrid alignment...")
seq1 = input("Enter the first sequence: ")
seq2 = input("Enter the second sequence: ")
gap_penalty_consecutive = -int(input("Enter the gap penalty for consecutive gaps: "))
gap_penalty_isolated = -int(input("Enter the gap penalty for isolated gaps: "))
match_gain = int(input("Enter the match gain: "))
mismatch_loss = -int(input("Enter the mismatch loss: "))
score, aligned_seq1, aligned_seq2 = hybrid_alignment(seq1, seq2, gap_penalty_consecutive, gap_penalty_isolated, match_gain, mismatch_loss)
print("Optimal Alignment Score:", score)
print("Aligned Sequence 1:", aligned_seq1)
print("Aligned Sequence 2:", aligned_seq2)

print("------------------------------------------------------------")

print("Executing exact pattern matching using BWT...")
dna_sequence = input("Enter the DNA sequence: ")
patterns_input = input("Enter the DNA patterns (comma-separated): ")
patterns = patterns_input.split(",")

dna_sequence = dna_sequence + '$'
bwt, suffix_array, sorted_rotations = bwt_transform(dna_sequence)

last_to_first = last_to_first_mapping(bwt, suffix_array)

print("\nBWT:", bwt)
print("Suffix Array:", suffix_array)
print("Sorted Rotations:")
for i, rotation in enumerate(sorted_rotations):
    print(f"{i}: {rotation}")
print("Last-to-First Mapping:", last_to_first)

match_positions = find_exact_matches(bwt, suffix_array, last_to_first, patterns, dna_sequence)

print("\nExact Pattern Matches:")
for pattern, positions in match_positions.items():
    print(f"Pattern: {pattern}, Positions: {positions}")

print("------------------------------------------------------------")

print("Executing Smith-Waterman with Compression...")
seq1 = input("Enter the first DNA sequence: ")
seq2 = input("Enter the second DNA sequence: ")
optimal_score, decompressed_seq1, aligned_seq2 = smith_waterman_with_compression(seq1, seq2)
print("Optimal Alignment Score:", optimal_score)
print("Decompressed and Aligned Sequence 1:", decompressed_seq1)
print("Aligned Sequence 2:", aligned_seq2)

Executing optimized edit distance with gap penalties...
The minimum edit distance with gap penalties is: 4
------------------------------------------------------------
Executing hybrid alignment...
Optimal Alignment Score: 5
Aligned Sequence 1: GATTACA
Aligned Sequence 2: GCATGCU
------------------------------------------------------------
Executing exact pattern matching using BWT...

BWT: ACTGA$TA
Suffix Array: [7, 6, 4, 1, 5, 0, 3, 2]
Sorted Rotations:
0: $GATTACA
1: A$GATTAC
2: ACA$GATT
3: ATTACA$G
4: CA$GATTA
5: GATTACA$
6: TACA$GAT
7: TTACA$GA
Last-to-First Mapping: [6, 5, 3, 0, 4, 7, 2, 1]

Exact Pattern Matches:
Pattern: GA, Positions: [1]
Pattern: TTA, Positions: [3, 4]
Pattern: AC, Positions: [7, 5, 2]
------------------------------------------------------------
Executing Smith-Waterman with Compression...
Optimal Alignment Score: 0
Decompressed and Aligned Sequence 1: GATTACAATTACAGTTACAGATACAGATACAGATTCAGATTAAGATTAC
Aligned Sequence 2: ATGCU
